In [28]:
import pandas as pd
import numpy as np
import hashlib
from urllib.parse import urlparse

In [3]:
pd.__version__

'1.3.5'

In [4]:
filename = 'data/2021_12_30_news/2021_12_30_eluniversal_articles.csv'

In [35]:
df = pd.read_csv(filename)

In [36]:
df.head()

,body,date,image,location,news_site,time,title,url
0,"\nNo acción individual ni cuentapropismo, sí u...",NaN,NaN,NaN,eluniversal,NaN,NaN,https://www.eluniversal.com.mx/opinion/alejand...
1,"\n Con la puesta en marcha de la NOM-035, los ...",30/12/2021,NaN,Cartera,eluniversal,\n 02:46,El estrés financiero limita productividad de e...,https://www.eluniversal.com.mx/cartera/el-estr...
2,\n\nLleva más de 450 millones de años en el pl...,NaN,NaN,NaN,eluniversal,NaN,NaN,https://www.eluniversal.com.mx/opinion/adriana...
3,"\n\nDurante 2021, los paisanos se han converti...",30/12/2021,https://www.eluniversal.com.mx/sites/default/f...,Cartera,eluniversal,\n 02:48,"Paisanos, personajes del año por récord en rem...",https://www.eluniversal.com.mx/cartera/paisano...
4,\n\nLo dicho por el padre Solalinde no es una ...,29/12/2021,https://www.eluniversal.com.mx/sites/default/f...,Nación,eluniversal,\n 02:56,"¿San Andrés Manuel, el ratificado?",https://www.eluniversal.com.mx/opinion/ricardo...


In [7]:
df.shape

(30, 8)

In [8]:
df = df.convert_dtypes()

In [9]:
df.dtypes

body         string
date         string
image        string
location     string
news_site    string
time         string
title        string
url          string
dtype: object

In [12]:
def _extract_host(df):
    df['host'] = df['url'].apply(lambda url: urlparse(url).netloc)

In [37]:
_extract_host(df)
df.sample(5)

,body,date,image,location,news_site,time,title,url,host
13,\n\nLos comicios libres y transparentes para e...,29/12/2021,https://www.eluniversal.com.mx/sites/default/f...,Nación,eluniversal,\n 02:50,Democracia sindical sin farsas,https://www.eluniversal.com.mx/opinion/editori...,www.eluniversal.com.mx
7,\n Luego de posponer el concierto de fin de añ...,30/12/2021,NaN,Metrópoli,eluniversal,\n 02:46,"Alcaldías de CDMX, libres de cancelar eventos ...",https://www.eluniversal.com.mx/metropoli/alcal...,www.eluniversal.com.mx
27,\n\nNi el llamado del presidente Andrés Manuel...,29/12/2021,https://www.eluniversal.com.mx/sites/default/f...,Nación,eluniversal,\n 02:50,Presionan y operan en Pemex para imponer en si...,https://www.eluniversal.com.mx/nacion/presiona...,www.eluniversal.com.mx
20,\n\nEstamos a unos días de que 2021 concluya. ...,29/12/2021,https://www.eluniversal.com.mx/sites/default/f...,Nación,eluniversal,\n 02:52,Los retos de 2022,https://www.eluniversal.com.mx/opinion/alfonso...,www.eluniversal.com.mx
4,\n\nLo dicho por el padre Solalinde no es una ...,29/12/2021,https://www.eluniversal.com.mx/sites/default/f...,Nación,eluniversal,\n 02:56,"¿San Andrés Manuel, el ratificado?",https://www.eluniversal.com.mx/opinion/ricardo...,www.eluniversal.com.mx


In [38]:
def _fill_missing_titles(df):
    missing_titles_mask = df['title'].isna()
    missing_titles = df[missing_titles_mask]['url'].str.extract(r'(?P<missing_titles>[^/]+)$').applymap(lambda title: title.split('-')).applymap(lambda title_words: ' '.join(title_words))
    df.loc[missing_titles_mask, 'title'] = missing_titles.loc[:, 'missing_titles']
    return df

In [39]:
df = _fill_missing_titles(df)
df.loc[missing_titles_mask, 'title'].head()

0     vigilar armar la causa castigar foucault revis...
2                      brian nissen y el fosil viviente
6                                        feliz ano 2022
10                              ascenso si repechaje no
11                                     caceria politica
Name: title, dtype: object

In [40]:
def _generate_row_uids(df):
    uids = df.apply(lambda row: hashlib.md5(bytes(row['url'].encode())), axis=1).apply(lambda url_hashed: url_hashed.hexdigest())
    df['uid'] = uids
    return df


In [43]:
df = _generate_row_uids(df)
df.sample(5)

,body,date,image,location,news_site,time,title,url,host,uid
22,\n El incremento de contagios de Covid-19 regi...,30/12/2021,https://www.eluniversal.com.mx/sites/default/f...,Metrópoli,eluniversal,\n 02:44,Se reanuda el peregrinaje por pruebas de Covid...,https://www.eluniversal.com.mx/metropoli/se-re...,www.eluniversal.com.mx,20bf3e2bfed0120fb1b424d8f08e9a2b
14,\nCiudad Mier al atardecer: es la hora en la q...,NaN,NaN,NaN,eluniversal,NaN,tierra sin ley,https://www.eluniversal.com.mx/opinion/hector-...,www.eluniversal.com.mx,131b739131f249272a961704f2a430ae
1,"\n Con la puesta en marcha de la NOM-035, los ...",30/12/2021,NaN,Cartera,eluniversal,\n 02:46,El estrés financiero limita productividad de e...,https://www.eluniversal.com.mx/cartera/el-estr...,www.eluniversal.com.mx,a89121eef7d0aa55d4bd5ab5f3ac03c7
29,"\n Otumba, Méx.— Una persona murió y otra más ...",30/12/2021,https://www.eluniversal.com.mx/sites/default/f...,Metrópoli,eluniversal,\n 02:46,Explota taller de pirotecnia en Otumba; muere 1,https://www.eluniversal.com.mx/metropoli/explo...,www.eluniversal.com.mx,6addef721a83367b51db9d23203f07b0
23,\n\nMadrid.- La salud del planeta se deteriora...,29/12/2021,https://www.eluniversal.com.mx/sites/default/f...,Mundo,eluniversal,\n 00:53,"Cambio climático, el mundo en emergencia",https://www.eluniversal.com.mx/mundo/cambio-cl...,www.eluniversal.com.mx,76a0e4a329293b9b2fc61431c1f3cb64


,body,date,image,location,news_site,time,title,url,host,uid
12,"\nEl que de nuevo brilló como “rockstar”, aunq...",29/12/2021,https://www.eluniversal.com.mx/sites/default/f...,Nación,eluniversal,\n 03:00,El pararrayos López-Gatell,https://www.eluniversal.com.mx/opinion/periodi...,www.eluniversal.com.mx,c08eaeaaac45402aea566cdc90388a1e
27,\n\nNi el llamado del presidente Andrés Manuel...,29/12/2021,https://www.eluniversal.com.mx/sites/default/f...,Nación,eluniversal,\n 02:50,Presionan y operan en Pemex para imponer en si...,https://www.eluniversal.com.mx/nacion/presiona...,www.eluniversal.com.mx,df15c87d230ad1639fa2a04a6c3c8d8f
13,\n\nLos comicios libres y transparentes para e...,29/12/2021,https://www.eluniversal.com.mx/sites/default/f...,Nación,eluniversal,\n 02:50,Democracia sindical sin farsas,https://www.eluniversal.com.mx/opinion/editori...,www.eluniversal.com.mx,b690b7ec2443f3bdbdafe62940afde37
9,"\n La jefa de Gobierno de la Ciudad de México,...",30/12/2021,https://www.eluniversal.com.mx/sites/default/f...,Metrópoli,eluniversal,\n 02:45,“CDMX cierra año con baja de 40% en delitos”,https://www.eluniversal.com.mx/metropoli/cdmx-...,www.eluniversal.com.mx,011e5301dccf185e32885b066e2da57c
1,"\n Con la puesta en marcha de la NOM-035, los ...",30/12/2021,NaN,Cartera,eluniversal,\n 02:46,El estrés financiero limita productividad de e...,https://www.eluniversal.com.mx/cartera/el-estr...,www.eluniversal.com.mx,a89121eef7d0aa55d4bd5ab5f3ac03c7
